# <center> 1.8 - tqdm, pandarallel</center>

tqdm - библиотека, добавляющая в вывод ячеек красивые прогресс-бары.

In [15]:
import pandas as pd
from tqdm.notebook import tqdm
import time

In [19]:
# Возьмем датасет car_info
car_info = pd.read_csv('../data/car_info.csv')
print(car_info.shape)
car_info.sample(5)

(1500, 10)


,car_type,fuel_type,car_rating,year_to_start,riders,car_id,model,target_2,year_to_work,target_1
1031,economy,petrol,0.4,2016,27805,z52947624y,Smart Coupe,wheel_shake,2017,3.942265e+05
1102,business,petrol,0.1,2016,29552,R-6980341r,BMW 320i,break_bug,2020,3.886251e+06
1195,standart,petrol,-1.0,2014,8810,O-1357458W,Renault Kaptur,gear_stick,2015,2.213449e+06
267,economy,petrol,1.7,2016,33320,o54272769c,Smart Coupe,another_bug,2020,3.084599e+06
922,economy,petrol,4.4,2015,6053,t10563344V,Kia Rio,engine_overheat,2021,2.500602e+06


In [5]:
car_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_type       1500 non-null   object 
 1   fuel_type      1500 non-null   object 
 2   car_rating     1500 non-null   float64
 3   year_to_start  1500 non-null   int64  
 4   riders         1500 non-null   int64  
 5   car_id         1500 non-null   object 
 6   model          1500 non-null   object 
 7   target_2       1500 non-null   object 
 8   year_to_work   1500 non-null   int64  
 9   target_1       1500 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 117.3+ KB


Напишем функцию, которая будет хранить столбцы датафрейма в оптимальных типах, что может существенно уменьшить объём потребляемой памяти.

In [9]:
def optimize_df(df):
    types = list()
    for x in df.dtypes.tolist():
        if x=='int64':
            types.append('int16')
        elif x=='float64':
            types.append('float32')
        else:
            types.append('object')
    types = dict(zip(df.columns.tolist(),types))
    return df.astype(types)

In [10]:
# Запустим на нашем датасете
car_info = optimize_df(car_info)

Непонятно отработала функция или нет и сколько ждать до её окончания? Особенно актуально на больших датасетах с тысячами столбцов. <br> 
Добавим прогресс бар.

In [16]:
def progress_optimize_df(df):
    types = list()
    for x in tqdm(df.dtypes.tolist()):
        if x=='int64':
            types.append('int16')
        elif x=='float64':
            types.append('float32')
        else:
            types.append('object')
        time.sleep(1) # добавим паузу 1сек. для наглядности

    types = dict(zip(df.columns.tolist(),types))
    return df.astype(types)

In [20]:
car_info = pd.read_csv('../data/car_info.csv')
car_info = progress_optimize_df(car_info)

  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
car_info.info() # Объем занимаемой памяти уменьшился на 30%

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   car_type       1500 non-null   object 
 1   fuel_type      1500 non-null   object 
 2   car_rating     1500 non-null   float32
 3   year_to_start  1500 non-null   int16  
 4   riders         1500 non-null   int16  
 5   car_id         1500 non-null   object 
 6   model          1500 non-null   object 
 7   target_2       1500 non-null   object 
 8   year_to_work   1500 non-null   int16  
 9   target_1       1500 non-null   float32
dtypes: float32(2), int16(3), object(5)
memory usage: 79.2+ KB


Теперь допустим мы захотели проитерироваться по всему нашему датасету и посчитать сколько было водителей на всех машинах за все время. 

In [24]:
riders = 0
for ind, row in tqdm(car_info.iterrows()):
    time.sleep(0.01)
    riders += row['riders']

0it [00:00, ?it/s]

Видно, что функция не знает сколько всего будет объектов и сколько времени до конца осталось тоже посчитать не может. Так как <code>iterrows()</code>
выдает ряд датафрейма и его индекс. В этом случае функции <code>tqdm()</code> надо указать сколько всего их будет в параметре <code>total</code>

In [25]:
riders = 0
for ind, row in tqdm(car_info.iterrows(), total=len(car_info)):
    riders += row['riders']
    time.sleep(0.01)
riders
# теперь прогресс-бар отображается как нужно

  0%|          | 0/1500 [00:00<?, ?it/s]

6642500

## <center> progress_apply()</center>

А теперь мы внезапно поняли, что название модели автомобиля очень информативный столбец и решили на его основе нагенерировать странных признаков:

In [27]:
car_info['char_count'] = car_info['model'].apply(len)
car_info['word_count'] = car_info['model'].apply(lambda x: len(x.split()))
car_info['different_words'] = car_info['model'].apply(lambda x: len(set(x.split())))
car_info['title_word_count'] = car_info['model'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
car_info['upper_case_word_count'] = car_info['model'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
car_info['is_bmw'] = car_info['model'].apply(lambda x: 1 if 'BMW' in x else 0)
car_info['is_kia'] = car_info['model'].apply(lambda x: 1 if 'Kia' in x else 0)
car_info['is_smart'] = car_info['model'].apply(lambda x: 1 if 'Smart' in x else 0)

Снова непонятно сколько ждать и идет ли процесс. Давайте добавим прогресс-бар прям в пандас. <br> Для этого немного по другому импортируем функцию <code>tqdm()</code>

In [29]:
from tqdm import tqdm
tqdm.pandas()

#и просто заменяем apply на progress_apply(), тоже самое можно сделать для df.map()
car_info['char_count'] = car_info['model'].progress_apply(len)
car_info['word_count'] = car_info['model'].progress_apply(lambda x: len(x.split()))
car_info['different_words'] = car_info['model'].progress_apply(lambda x: len(set(x.split())))
car_info['title_word_count'] = car_info['model'].progress_apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
car_info['upper_case_word_count'] = car_info['model'].progress_apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
car_info['is_bmw'] = car_info['model'].progress_apply(lambda x: 1 if 'BMW' in x else 0)
car_info['is_kia'] = car_info['model'].progress_apply(lambda x: 1 if 'Kia' in x else 0)
car_info['is_smart'] = car_info['model'].progress_apply(lambda x: 1 if 'Smart' in x else 0)
car_info # проверим, что новые признаки появились

100%|██████████| 1500/1500 [00:00<00:00, 1211059.87it/s]


,car_type,fuel_type,car_rating,year_to_start,riders,car_id,model,target_2,year_to_work,target_1,char_count,word_count,different_words,title_word_count,upper_case_word_count,is_bmw,is_kia,is_smart
0,economy,petrol,-0.9,2013,22178,s-6180865X,Smart ForFour,engine_overheat,2017,2.342619e+06,13,2,2,1,0,0,0,1
1,economy,petrol,-0.6,2014,479,L-1050707e,Smart ForFour,engine_overheat,2019,5.614556e+05,13,2,2,1,0,0,0,1
2,economy,petrol,1.3,2015,12402,u10315229l,VW Polo VI,break_bug,2019,1.838453e+06,10,3,3,1,2,0,0,0
3,business,electro,1.2,2014,11318,b-1247294W,MINI CooperSE,engine_fuel,2021,2.863095e+06,13,2,2,0,1,0,0,0
4,economy,petrol,-1.2,2014,28146,M-3209979P,VW Tiguan,wheel_shake,2019,2.138686e+06,9,2,2,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,economy,petrol,-1.0,2013,25137,s14346341L,Smart ForTwo,wheel_shake,2015,1.351647e+06,12,2,2,1,0,0,0,1
1496,economy,petrol,3.9,2017,-15271,v21225421p,Skoda Rapid,engine_check,2019,6.422310e+06,11,2,2,2,0,0,0,0
1497,economy,petrol,1.0,2012,-17243,T14866175p,VW Polo VI,break_bug,2016,9.535158e+05,10,3,3,1,2,0,0,0
1498,standart,petrol,1.4,2015,23191,j12230337z,Kia Sportage,another_bug,2018,3.840840e+06,12,2,2,2,0,0,1,0


## <center>pandarallel - ускоряем pandas в N раз</center>

Допустим мы написали функцию для генерации нового признака со сложной логикой, которая долго выполняется (сейчас заменим её паузой)

In [36]:
# посчитаем количество лет эксплуатации автомобиля на 2022год
def get_car_age(x):
    time.sleep(0.01)
    return 2022 - x

In [37]:
%%time
car_info['car_age'] = car_info['year_to_start'].progress_apply(get_car_age)

100%|██████████| 1500/1500 [00:15<00:00, 99.15it/s]

CPU times: user 50.3 ms, sys: 22.8 ms, total: 73.1 ms
Wall time: 15.1 s


А теперь давайте ускорим генерацию признака в n раз (n - количество ядер процессора), в нашем случае 4.

In [38]:
# импортируем библиотеку pandarallel и инициализируем её
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=4)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [39]:
%%time
# запускаем генерацию нашего признака в 4 потока, заменяя progress_apply() на parallel_apply()
car_info['car_age'] = car_info['year_to_start'].parallel_apply(get_car_age)

CPU times: user 74.3 ms, sys: 28 ms, total: 102 ms
Wall time: 3.84 s


С 15.1 сек, время уменьшилось до 3.84сек.

Библиотека pandarallel позволяет задействовать все доступные ядра процессора для параллельных вычислений. Если функции содержат в себе вызовы других функций или использование сложных фреймворков, то не всегда их удасться распараллелить. <br>
Так же можно распараллелить и другие функции пандас, полный список из <a href='https://nalepae.github.io/pandarallel/'> Github</a> проекта:

![sheet](14rxcddfc9lms2sdubj9h7zgykm.png)

<img src="https://habrastorage.org/r/w1560/webt/14/rx/cd/14rxcddfc9lms2sdubj9h7zgykm.png">